### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집 방법
- 동이름 > 매물정보 (데이터프레임)

- 절차
    - 동이름 입력받음 > 위도, 경도값 가져옴
    - 위도, 경도값 > geohash code(지역범위) 가져옴 - 지역범위 나타내는 데이터
    - geohash > 매물 아이디 가져옴
    - 매물 아이디 > 매물 정보

In [1]:
import pandas as pd
import requests

In [ ]:
# 1. 동이름 > 위도, 경도

In [6]:
query = '강서구 염창동'
url  = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸"
response = requests.get(url)
datas  =response.json()['items'][0]
# datas
lat,lng = datas["lat"], datas['lng']
lat,lng

(37.55374526977539, 126.87098693847656)

In [ ]:
# 2. 위도,경도 > geohash , #위도경도값을 geohash 로 바꿔주는 패키지 - geohash2

In [8]:
!pip install geohash2 

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=c0e275b348a51058a278108b5605b8c4764b7052806e84409abb469ad3fb09f9
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [9]:
import geohash2

In [12]:
# precision -> 영역: 값이 커질수록 영역이 작아짐
code= geohash2.encode(lat,lng, precision= 5)
code

'wydjx'

In [ ]:
# 3. geohash > 매물 아이디

In [14]:

url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [18]:
# response.text  -> response.json()
datas = response.json()["items"]
datas[:3]

[{'lat': 37.52911584984866, 'lng': 126.89761140063823, 'item_id': 30462357},
 {'lat': 37.529618523515886, 'lng': 126.89757894127727, 'item_id': 30524930},
 {'lat': 37.52911607377812, 'lng': 126.89697440047796, 'item_id': 30629925}]

In [20]:
# ids = []
# for data in datas:
#     ids.append(data['item_id'])
# ids[:5]

#list comprehention 쓰면 간결한 코드 가능 ->

# result = [data**2 for data in range(10) if data %2]

ids = [data['item_id'] for data in datas]
ids[:5]

[30462357, 30524930, 30629925, 30681505, 30494154]

In [ ]:
# 4. 매물 아이디 > 매물 정보

In [21]:
ids[:3]

[30462357, 30524930, 30629925]

In [22]:
url = "https://apis.zigbang.com/v2/items/list"
params = {"domain": "zigbang",
          "item_ids": ids,
         "withCoalition": "true",
         }
response = requests.post(url, params)
response

<Response [200]>

In [34]:
items = response.json()["items"]
item_df = pd.DataFrame(items)
# item_df.tail(2)
item_df

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,30462357,https://ic.zigbang.com/ic/items/30462357/1.jpg,전세,전세,8000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,2,2,8,"융자없는 풀옵션전세원룸 입주즉시가능 청소,도배완료",None,01,영등포구 당산동4가,"{'lat': 37.52960996198865, 'lng': 126.89699735...",False,True,원룸,[],서울시 영등포구 당산동4가,None,None,13,2022-02-18T17:43:12+09:00,False
1,None,30524930,https://ic.zigbang.com/ic/items/30524930/1.jpg,전세,전세,7000,0,16.53,"{'m2': 16.53, 'p': '5'}","{'m2': 16.53, 'p': '5'}",None,None,7,7,8,💕놓칠수없는가격💕파격EVENT💕조절가능💕즉입가능💕,None,01,영등포구 당산동4가,"{'lat': 37.52962024251799, 'lng': 126.89758341...",False,True,원룸,[추천],서울시 영등포구 당산동4가,None,None,13,2022-02-07T16:03:16+09:00,False
2,None,30629925,https://ic.zigbang.com/ic/items/30629925/1.jpg,전세,전세,7000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,7,7,8,🧡더블역세권💥착한전세가💥밝은주변환경💥융자❌💛,None,01,영등포구 당산동4가,"{'lat': 37.52960972721912, 'lng': 126.89759074...",False,True,원룸,[추천],서울시 영등포구 당산동4가,None,None,12,2022-02-19T16:52:02+09:00,True
3,None,30681505,https://ic.zigbang.com/ic/items/30681505/1.jpg,전세,전세,7000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,7,7,8,★실제매물입니다★ 영등포구청역 융자없는 깔끔한 원룸입니다.,None,01,영등포구 당산동4가,"{'lat': 37.52957802687841, 'lng': 126.89760100...",False,True,원룸,[추천],서울시 영등포구 당산동4가,None,None,13,2022-02-17T15:58:52+09:00,False
4,None,30494154,https://ic.zigbang.com/ic/items/30494154/1.jpg,전세,전세,6000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,5,5,8,※양평역 원룸 전세※ #전세대출불가 #대로변 #가성비,None,01,영등포구 양평동3가,"{'lat': 37.529143522124684, 'lng': 126.8913258...",False,True,원룸,[],서울시 영등포구 양평동3가,None,None,12,2022-02-05T11:14:17+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,None,30653716,https://ic.zigbang.com/ic/items/30653716/1.jpg,월세,월세,4000,27,15.00,"{'m2': 15, 'p': '4.5'}","{'m2': 15, 'p': '4.5'}",None,None,2,2,4,🐯성산동 신축급 풀옵션🐯,None,01,마포구 성산동,"{'lat': 37.568536060504684, 'lng': 126.9089773...",False,True,원룸,[],서울시 마포구 성산동,None,None,8,2022-02-19T12:13:37+09:00,False
912,None,30673351,https://ic.zigbang.com/ic/items/30673351/1.jpg,월세,월세,9000,20,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,1,1,4,🌿 테라스 있는 반전세 원룸 / 월세 가능 / 신축 🌿,None,01,마포구 성산동,"{'lat': 37.56807358994841, 'lng': 126.90896423...",False,True,원룸,[추천],서울시 마포구 성산동,None,None,8,2022-02-17T10:27:46+09:00,False
913,None,30629059,https://ic.zigbang.com/ic/items/30629059/1.jpg,전세,전세,26000,0,65.00,"{'m2': 65, 'p': '19.7'}","{'m2': 60, 'p': '18.2'}",None,None,3,3,3,올리모델링한 전용 넓은 쓰리룸 전세,None,05,마포구 성산동,"{'lat': 37.56852662356126, 'lng': 126.90707409...",False,True,빌라,[],서울시 마포구 성산동,None,None,0,2022-02-14T14:50:13+09:00,False
914,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}",None,None,6,6,6,🔶🔶마포 저렴하고 전망 좋은 복층 쓰리룸 탑층🔶🔶,None,05,마포구 중동,"{'lat': 37.56946731656015, 'lng': 126.90886533...",False,True,빌라,[추천],서울시 마포구 중동,None,None,3,2022-02-03T12:00:40+09:00,False


In [35]:
pd.options.display.max_columns =30
pd.options.display.max_rows

60

In [36]:
item_df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new'],
      dtype='object')

In [41]:
columns = ['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적','address']

In [40]:
query

'강서구 염창동'

In [39]:
result_df = item_df[columns]
result_df
result_df = result_df

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적
0,None,30462357,https://ic.zigbang.com/ic/items/30462357/1.jpg,전세,전세,8000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}"
1,None,30524930,https://ic.zigbang.com/ic/items/30524930/1.jpg,전세,전세,7000,0,16.53,"{'m2': 16.53, 'p': '5'}","{'m2': 16.53, 'p': '5'}"
2,None,30629925,https://ic.zigbang.com/ic/items/30629925/1.jpg,전세,전세,7000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}"
3,None,30681505,https://ic.zigbang.com/ic/items/30681505/1.jpg,전세,전세,7000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}"
4,None,30494154,https://ic.zigbang.com/ic/items/30494154/1.jpg,전세,전세,6000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}"
...,...,...,...,...,...,...,...,...,...,...
911,None,30653716,https://ic.zigbang.com/ic/items/30653716/1.jpg,월세,월세,4000,27,15.00,"{'m2': 15, 'p': '4.5'}","{'m2': 15, 'p': '4.5'}"
912,None,30673351,https://ic.zigbang.com/ic/items/30673351/1.jpg,월세,월세,9000,20,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}"
913,None,30629059,https://ic.zigbang.com/ic/items/30629059/1.jpg,전세,전세,26000,0,65.00,"{'m2': 65, 'p': '19.7'}","{'m2': 60, 'p': '18.2'}"
914,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}"


In [29]:
item_dep_size = pd.DataFrame(items)[["deposit","size_m2"]]

In [31]:
item_dep_size.tail()

,deposit,size_m2
911,4000,15.00
912,9000,19.83
913,26000,65.00
914,28000,33.58
915,5000,41.36


In [ ]:
#보증금 1억이하, 월세 100만원 이하
result_df[(result_df['deposit']<=10000)& (result_df['rent']<=100)]

In [ ]:
def oneroom(address):
    url  = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
    response = requests.get(url)
    datas  =response.json()['items'][0]
    lat,lng = datas["lat"], datas['lng']
    
    code= geohash2.encode(lat,lng, precision= 5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    
    datas = response.json()["items"]
    ids = [data['item_id'] for data in datas]
    return 